# GUM Uncertainty Calculation with Python

Exploring uncertainty calculation and testing examples in the official GUM (**G**uide to the expression of **u**ncertainty in **m**easurement) document ([https://doi.org/10.59161/JCGM100-2008E](https://doi.org/10.59161/JCGM100-2008E)) using Python, particularly the [uncertainties](https://pythonhosted.org/uncertainties/index.html) package.

Packages installed (you may change `mamba` to `conda`):  
```
mamba install -c conda-forge jupyterlab numpy matplotlib scienceplots watermark ipympl uncertainties scipy
```
Alternatively, use the provided `yml` file and change the `ENV_NAME` to your desire, e.g. `gum`:

```
mamba env create -n ENV_NAME -f environment.yml
````

This notebook should work top down by running each cell interactively. Alternatively, use "Run All Cells" once and read through the document; you can still play with the values in the code cells.

This notebook is based on this [basic template](https://github.com/lukmuk/jupyter-scientific-notebook-templates).

### Content
- [Notebook Settings](#Notebook-Settings)
- [Package Imports](#Package-Imports)
- [4.4.3](#4.4.3)
- [G.4.1](#G.4.1)
- [5.1.5](#5.1.5)
- [5.2.2](#5.2.2)
- [H.1](#H.1)
- [H.2](#H.2)
- [Export](#Export)
- [Markdown Snippets](#Markdown-Snippets)

### Notebook Settings
[[Top](#Content)]

In [1]:
# Choose plotting backend (widget. qt. inline. ...)
#%matplotlib widget 

# Use watermark package to document package versions and hardware info
%load_ext watermark

# Other metadata
author = "Lukas Grünewald"

### Package Imports
[[Top](#Content)]

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from pathlib import Path

import scipy.stats as st
import uncertainties as uc

import scienceplots # https://github.com/garrettj403/SciencePlots/wiki/Gallery
plt.style.use(['science', 'notebook'])

In [3]:
%watermark -d --iv -v -u -m -co -a "$author"

Author: Lukas Grünewald

Last updated: 2024-11-15

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

conda environment: gum

Compiler    : MSC v.1941 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 13, GenuineIntel
CPU cores   : 16
Architecture: 64bit

matplotlib   : 3.9.2
scienceplots : 2.1.1
numpy        : 2.1.3
uncertainties: 3.2.2
scipy        : 1.14.1



### 4.4.3
[[Top](#Content)]

In [4]:
# Temperature data in degree Celsius from Table 1
t = np.array([96.90,
              98.18, 98.25,
              98.61, 99.03, 99.49,
              99.56, 99.74, 99.89, 100.07, 100.33, 100.42,
              100.68, 100.95, 101.11, 101.20,
              101.57, 101.84, 102.36,
              102.72],
            )

In [5]:
# Mean / arithmetic average as best estimation
t_mean = np.mean(t)
t_mean

np.float64(100.145)

In [6]:
# Sample standard deviation
t_std = np.std(t, ddof=1) # ddof = 1 is required for Bessel's correction/reduction of degrees of freedom by 1
t_std

np.float64(1.4888444830595435)

<div class="alert alert-block alert-danger"><b>Beware:</b> For numpy's sample standard deviation function `np.std()`, we need to specify `ddof=1`.</div>

In [7]:
# Experimental standard deviation of the mean
n = t.size

t_u = t_std/np.sqrt(n)
t_u

np.float64(0.3329157472046673)

In [8]:
# where we used n measurements
n

20

In [9]:
# Same thing with scipy.stats.sem
# Notice that this function automatically uses the sample standard deviation whereas ddof=1 is required for np.std()
t_u = st.sem(t)
t_u

np.float64(0.3329157472046673)

<div class="alert alert-block alert-info"><b>Tip:</b> As noted in <b>B.2.17 Note 3</b>: The correct term  is actually <b>"Experimental standard deviation of the mean"</b> which is often incorrectly called "standard error of the mean". This is actually the case for the `scipy.stats.sem` function.</div>

**Expanded uncertainty (normal distribution)**   
$U(t) = k_p\cdot u(t)$

See also **Table G.1**.

For a given level of confidence $p$, we can caculate the coverage factor $k_p$:

In [10]:
# We can define a confidence level p and find the coverage factor kp
# Here exemplary for a normal distribution
# https://www.pythonclear.com/modules/norm-ppf/
p = 0.99

kp = st.norm.ppf((1 + p)/2)
kp

np.float64(2.5758293035489004)

See also [here](https://www.geo.fu-berlin.de/en/v/soga-py/Basics-of-statistics/Continous-Random-Variables/The-Standard-Normal-Distribution/Determining-the-z-Value/index.html) for a very nice explanation using scipy.

We can calculate the expanded uncertainty:

In [11]:
t_U = t_u * kp
t_U

np.float64(0.85753413726266)

Vice versa, as stated in **4.3.4**: The formula `st.norm.ppf((1 + p)/2)` can be used when a measurement instrument's uncertainty is stated as $p$ level of confidence to find the standard uncertainty $u$ from the expanded uncertainty $U$ by dividing with $k_p$.  
That is, if no other information is given, we can assume a normal distribution. 

In [12]:
# Find the lower and upper bounds of the confidence interval
lower_bound = t_mean - t_U
upper_bound = t_mean + t_U
# Print the confidence interval
print(f"The {p*100:.2f}% confidence interval of the mean {t_mean:.2f} is ({lower_bound:.2f}, {upper_bound:.2f}) assuming a normal distribution.")

The 99.00% confidence interval of the mean 100.14 is (99.29, 101.00) assuming a normal distribution.


We can invert the process and find the confidence level $p$ from a given $k_p$ (i.e., the area under the curve of a probability density function between -$k_p$ and $k_p$):

In [13]:
# We can also define kp and find the confidence level
# Again, with normal distribution as an example
kp = 1.96

p = st.norm.cdf(kp) - st.norm.cdf(-kp)
p

np.float64(0.950004209703559)

A good explanation about the interpretation of confidence intervalls: https://www.youtube.com/watch?v=5q2ac412hv4

**Expanded uncertainty (Student's $t$ distribution)** 

For a small number of measurements, it may be more appropriate to use the $t$ distribution.  
For large $n$, the [$t$ distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution) approximates the normal distribution. Therefore, it should be valid for small *and* large sample sizes $n$ (?).

$U(t) = k_p\cdot u(t) \approx t_p(v)\cdot u(t)$ with the effective degrees of freedom $v$.  
For $v\rightarrow\infty$ we get $t_p(v) \rightarrow k_p$.  

See also **Table G.2** in the GUM document.

In [14]:
# We can define a confidence level p and find the coverage factor kp = tp
# Here exemplary for a Student's t distribution for v degrees of freedom
p = 0.99
v = n-1

tp = st.t.ppf((1 + p)/2, df=v)
tp

np.float64(2.860934606449914)

See also [here](https://www.geo.fu-berlin.de/en/v/soga-py/Basics-of-statistics/Continous-Random-Variables/Students-t-Distribution/Students-t-Distribution-in-Python/index.html), similar to above.

In [15]:
t_U = t_u * tp
t_U

np.float64(0.952450182209964)

In [16]:
# Find the lower and upper bounds of the confidence interval
lower_bound = t_mean - t_U
upper_bound = t_mean + t_U
# Print the confidence interval
print(f"The {p*100:.2f}% confidence interval of the mean {t_mean:.2f} is ({lower_bound:.2f}, {upper_bound:.2f}) assuming a Student's t distribution with {n-1} effective degrees of freedom.")

The 99.00% confidence interval of the mean 100.14 is (99.19, 101.10) assuming a Student's t distribution with 19 effective degrees of freedom.


As for the normal distribution, we can invert the process and find the confidence level $p$ from a given $t_p$ (i.e., the area under the curve of a probability density function between -$t_p$ and $t_p$):

In [17]:
# We can also define tp and find the confidence level
tp = 2.86

p = st.t.cdf(tp, df=v) - st.t.cdf(-tp, df=v)
p

np.float64(0.9899795103086788)

### G.4.1
[[Top](#Content)]

For a measurand calculated from multiple measured properties, the effective degrees of freedom $v_{eff}$ of the measurand can be calculated  from **Welch-Satterthwaite** formula (assumes uncorrelated variables).  
The $v_{eff}$ is then used to calculate the coverage factor for the confidence intervall for a distribution (often normal or $t$).


In [18]:
# page 74 example
x1_u_rel = 0.25
x2_u_rel = 0.57
x3_u_rel = 0.82

x1_n = 10
x2_n = 5
x3_n = 15

x_u_rel = np.array([x1_u_rel, x2_u_rel, x3_u_rel])
x_n = np.array([x1_n, x2_n, x3_n])

#top = (x1_u_rel)**2 + (x2_u_rel)**2 + (x3_u_rel)**2
#top = np.sqrt(np.sum(x_u_rel**2))**4
#bot = np.sum(x_u_rel**4/(x_n-1))

In [19]:
v_eff = np.sum(x_u_rel**2)**2 / np.sum(x_u_rel**4/(x_n-1))
v_eff

np.float64(18.998742314267957)

In [20]:
# Welch-Satterthwaite formula, uncorrelated
def v_eff(u, v, return_int = True):
    if return_int:
        return int(np.floor(np.sum(u**2)**2/np.sum(u**4/v)))
    else:
        return np.sum(u**2)**2/np.sum(u**4/v)

<div class="alert alert-block alert-info"><b>Tip:</b> For the correlated case, see <a href="http://www.isgmax.com/Articles_Papers/Welch-Satterthwaite%20for%20Correlated%20Errors.pdf">here</a> (<i>A Welch-Satterthwaite Relation for Correlated Errors</i>, by Dr. Howard Castrup).</div>

### 5.1.5
[[Top](#Content)]  

**Uncorrelated** input quantities.

Here we use the [uncertainties](https://pythonhosted.org/uncertainties/index.html) package.

In [21]:
# all values in V
# measurand M = V + dV with the uncertainties u
# We replaced original V for M here
V = 0.928571
V_u = 12e-6

dV = 0
dV_u = 8.7e-6

In [22]:
from uncertainties import ufloat

In [23]:
Vu = ufloat(V, V_u)
dVu = ufloat(dV, dV_u)

In [24]:
M = Vu + dVu

In [25]:
M

0.928571+/-1.4821943192442752e-05

In [26]:
print(f'Automatic number of digits on the uncertainty using `uncertainties` package: M = ({format(M)}) V')

Automatic number of digits on the uncertainty using `uncertainties` package: M = (0.928571+/-0.000015) V


In [27]:
print('Define number of digits d using `:.du`, e.g. d=3: M = ({:.3u}) V'.format(M))

Define number of digits d using `:.du`, e.g. d=3: M = (0.9285710+/-0.0000148) V


In [28]:
print(f"The combined standard uncertainty u_c(M) is around {M.s*1e6:.0f} µV.")

The combined standard uncertainty u_c(M) is around 15 µV.


In [29]:
# This is the same value as M.s
np.sqrt(12e-6**2 + 8.7e-6**2)

np.float64(1.4821943192442752e-05)

### 5.2.2
[[Top](#Content)]  

**Correlated** input quantities.

Here we use the [uncertainties](https://pythonhosted.org/uncertainties/index.html) package.

In [30]:
from uncertainties import ufloat

In [31]:
Rs = ufloat(1000, 100e-3) # calibration resistor with uncertainty

# the calibration factor a is approximately 1, see F.1.2.3, example 2 - 
# but now Ri depends on Rs and the error is considered in further calculations
a = 1 
Ri = a*Rs # we assume each of the ten resistors is calibrated using Rs

In [32]:
Ri

1000.0+/-0.1

In [33]:
# Calculate Rref from ten Ri in series
Rref = 10*Ri

In [34]:
print(f'Rref = ({format(Rref)}) Ohm')

Rref = (10000.0+/-1.0) Ohm


Note: This uncertainty of *correlated* variables $R_i$ is different than taking the root of the summed square of 100 mV errors (= assumption of *uncorrelated* $R_i$)

In [35]:
# Note: This uncertainty of correlated variables Ri is different than taking the root of the summed square of 100 mV errors (= uncorrelated Ri)
np.sqrt((100e-3**2) * 10) # Ohm

np.float64(0.31622776601683794)

which we could have achieved by defining different Ri, e.g. for 2:

In [36]:
# Uncorrelated
Ri0 = ufloat(1000, 100e-3)
Ri1 = ufloat(1000, 100e-3)

Ri0 + Ri1

2000.0+/-0.14142135623730953

In [37]:
# Correlated
2*Ri0

2000.0+/-0.2

### H.1
[[Top](#Content)]  

Example for **uncorrelated** input variables.

We can collect all values and uncertainties (**Table H.1**) and calculate the desired property: 

In [38]:
lSu = ufloat(50.000623e-3, 25e-9)
du = ufloat(215e-9, 9.7e-9)
asu = ufloat(11.5e-6, 1.2e-6)
dau = ufloat(0, 0.58e-6)
dthetau = ufloat(0, 0.029)
thetau = ufloat(-0.1, 0.41)

In [39]:
# calculate desired property
lu = lSu + du - lSu*(dau*thetau + asu*dthetau)

In [40]:
lu

0.050000838+/-3.171060964043185e-08

In [41]:
print('l = ({:.2u}) mm'.format(lu*1e3))

l = (50.000838+/-0.000032) mm


Note, that we can use scipy to find uncertainty value, e.g., for the $t$ distribution in **H.1.3.2**:

In [42]:
# Find t_p(v) factor for t_95(v=5)
p = 0.95
n = 6
v = n-1

tp = st.t.ppf((1 + p)/2, df=v)
tp

np.float64(2.570581835636314)

In [43]:
print(f'u(d1) = {0.01} µm/{tp:.2f} = {0.01/tp*1e3:.1f} nm')

u(d1) = 0.01 µm/2.57 = 3.9 nm


We continue with finding a expanded uncertainty for $p=0.99$. This requires knowledge of the degrees of freedom.

In [44]:
# Welch-Satterthwaite formula, uncorrelated
def v_eff(u, v, return_int = True):
    if return_int:
        return int(np.floor(np.sum(u**2)**2/np.sum(u**4/v)))
    else:
        return np.sum(u**2)**2/np.sum(u**4/v)

In [45]:
#def v_Btype(unc_val):
#    return 0.5*(unc_val.s/unc_val.n)**(-2)

In [46]:
d2 = np.array([5.8, 3.9, 6.7]) # standard uncertainties d, d1, d2
v2 = np.array([24, 5, 8]) # degrees of freedom

In [47]:
# Calculate effective degrees of freedom
v_eff(d2, v2)

25

In [48]:
d3 = np.array([lSu.s, du.s, 2.9e-9, 16.6e-9]) # standard uncertainties
v3 = np.array([18, v_eff(d2, v2), 50, 2]) # degrees of freedom

In [49]:
# Calculate effective degrees of freedom
v_eff(d3, v3)

16

In [50]:
p = 0.99
df = v_eff(d3, v3)

tp = st.t.ppf((1 + p)/2, df=df)
tp

np.float64(2.920781622496036)

In [51]:
lu_exp = ufloat(lu.n, lu.s*tp)

In [52]:
lu_exp

0.050000838+/-9.261976587591897e-08

We can report the final result as in the document, page 84:

In [53]:
print('l = ({:.2u}) mm, '.format(lu_exp*1e3))
print(f'where the number following the symbol ± is the numerical value of \
an expanded uncertainty U = k*uc, with U determined from a combined standard \
uncertainty uc = {lu.s*1e9:.0f} nm and a coverage factor k = {tp:.2f} based on \
the t-distribution for v = {df:.0f} degrees of freedom, and defines \
an interval estimated to have a level of confidence of {p*100} percent. \
The corresponding relative expanded uncertainty is U/l = {lu_exp.s/lu.n*1e6:.1f}*10^(-6)')

l = (50.000838+/-0.000093) mm, 
where the number following the symbol ± is the numerical value of an expanded uncertainty U = k*uc, with U determined from a combined standard uncertainty uc = 32 nm and a coverage factor k = 2.92 based on the t-distribution for v = 16 degrees of freedom, and defines an interval estimated to have a level of confidence of 99.0 percent. The corresponding relative expanded uncertainty is U/l = 1.9*10^(-6)


### H.2
[[Top](#Content)]  

Example for **correlated** input variables.


#### Approach 1: Average measurement values, then propagate uncertainties
Let's start by writing down and calculating the values in **Table H.2**:

In [54]:
# Measurement values from Table H.2
V = np.array([5.007, 4.994, 5.005, 4.990, 4.999])
I = np.array([19.663, 19.639, 19.640, 19.685, 19.678])
phi = np.array([1.0456, 1.0438, 1.0468, 1.0428, 1.0433])

In [55]:
from uncertainties import ufloat

In [56]:
# Type A uncertainties
Vu = ufloat(np.mean(V), st.sem(V))
Iu = ufloat(np.mean(I), st.sem(I))
phiu = ufloat(np.mean(phi), st.sem(phi))

In [57]:
print(f'V = ({format(Vu)}) V')
print(f'I = ({format(Iu)}) mA')
print(f'phi = ({format(phiu)}) rad')

V = (4.9990+/-0.0032) V
I = (19.661+/-0.009) mA
phi = (1.0445+/-0.0008) rad


In [58]:
# Correlation coefficients
r_VI = st.pearsonr(V,I).statistic
print(f'r_VI: {r_VI:.2f}')

r_Vphi = st.pearsonr(V,phi).statistic
print(f'r_Vphi: {r_Vphi:.2f}')

r_Iphi = st.pearsonr(I,phi).statistic
print(f'r_Iphi: {r_Iphi:.2f}')

r_VI: -0.36
r_Vphi: 0.86
r_Iphi: -0.65


Alternatively provide the full correlation coefficient matrix using numpy

In [59]:
# Alternatively provide the full correlation coefficient matrix using numpy
np.corrcoef([V,I,phi])

array([[ 1.        , -0.35531122,  0.85762421],
       [-0.35531122,  1.        , -0.64511122],
       [ 0.85762421, -0.64511122,  1.        ]])

In [60]:
# Note that this is not the same as the covariance matrix!
#np.cov([V,I,phi])

If we **do not take care** of the correlation coefficients, we will assume no correlation and underestimate the combined uncertainty.  
Here exemplary done for $Z = U/I$.

In [61]:
from uncertainties.umath import * 

In [62]:
Zu = Vu/Iu * 1e3 # in Ohms

In [63]:
print('Z = ({:.3u}) Ohm (uncorrelated case)'.format(Zu))

Z = (254.260+/-0.204) Ohm (uncorrelated case)


This is the case for the values shown in **Table H.5**.

We instead want to use the correlation coefficients above (`pearsonr`) to get the combined uncertainty for the correlated case.  
Here is the *manual* way for Z:

In [64]:
# Manual calculation according to equation H.8c
# relative uncertainty
Z_u_rel = np.sqrt( (Vu.s/Vu.n)**2 + (Iu.s/Iu.n)**2 - 2*(Vu.s/Vu.n)*(Iu.s/Iu.n)*r_VI)
Z_u_rel

np.float64(0.000929506832076323)

In [65]:
Z_u = Z_u_rel*Zu.n

In [66]:
Z_u

np.float64(0.23633613008237322)

In [67]:
Zu = ufloat(Zu.n, Z_u)

In [68]:
print('Zu = ({:.3u}) Ohm'.format(Zu))

Zu = (254.260+/-0.236) Ohm


Note, that the combined uncertainty increased from 0.204 (uncorrelated) to 0.236 (correlated).

Since manual calculation is tedious, we can let the `uncertainties` package handle the calculations.  
We only need the input values as numpy arrays and define all properties using `uncertainties.correlated_values_norm`:

In [69]:
import uncertainties

In [70]:
# Define as correlated variables
Vu, Iu, phiu = uncertainties.correlated_values_norm(
    [(np.mean(V), st.sem(V)), # define as value and uncertainties
     (np.mean(I), st.sem(I)), 
     (np.mean(phi), st.sem(phi))],
     np.corrcoef([V, I, phi]) # correlation coefficient matrix for correlated variables
)

In [71]:
# These are the same as before
print(f'V = ({format(Vu)}) V')
print(f'I = ({format(Iu)}) mA')
print(f'phi = ({format(phiu)}) rad')

V = (4.9990+/-0.0032) V
I = (19.661+/-0.009) mA
phi = (1.0445+/-0.0008) rad


As we defined the correlation coefficients among $V$, $I$, and $\phi$ we can simply calculate $R$, $X$, and $Z$:

In [72]:
# Calculate R, X, and Z
Ru = Vu/Iu*cos(phiu)*1e3
Xu = Vu/Iu*sin(phiu)*1e3
Zu = Vu/Iu*1e3

In [73]:
# Print out values to compare with Table H.3
print('R = ({:.2u}) Ohm'.format(Ru))
print('X = ({:.3u}) Ohm'.format(Xu))
print('Z = ({:.3u}) Ohm'.format(Zu))

R = (127.732+/-0.071) Ohm
X = (219.847+/-0.296) Ohm
Z = (254.260+/-0.236) Ohm


Note that these are the same combined uncertainties as shown in the [GUM PDF](https://doi.org/10.59161/JCGM100-2008E) **Table H.3**.

#### Approach 2: Calculate desired property from each set of measurements, average at the end
Let's start by writing down and calculating the values in **Table H.4**:

In [74]:
# Measurement values from Table H.2
V = np.array([5.007, 4.994, 5.005, 4.990, 4.999])
I = np.array([19.663, 19.639, 19.640, 19.685, 19.678])
phi = np.array([1.0456, 1.0438, 1.0468, 1.0428, 1.0433])

# Arrays with calculated values for R, X, and Z
R = V/I * np.cos(phi)
X = V/I * np.sin(phi)
Z = V/I

Note that R, X, and Z are arrays, e.g., R:

In [75]:
R

array([0.12767249, 0.12789245, 0.12750626, 0.12771042, 0.12787654])

We can now calculate the mean and standard deviation of the mean as usual.  
We define them here as uncertainty ufloat for convenience and display, but we do not need to perform any further calculation:

In [76]:
# Type A uncertainties
Ru = ufloat(np.mean(R), st.sem(R))
Xu = ufloat(np.mean(X), st.sem(X))
Zu = ufloat(np.mean(Z), st.sem(Z))

In [77]:
# Print out values to compare with Table H.3
print('R = ({:.2u}) Ohm'.format(Ru))
print('X = ({:.3u}) Ohm'.format(Xu))
print('Z = ({:.3u}) Ohm'.format(Zu))

R = (0.127732+/-0.000071) Ohm
X = (0.219847+/-0.000295) Ohm
Z = (0.254260+/-0.000236) Ohm


Note that these values are the same as the ones from combined uncertainty measurement with correlated input variables $V$, $I$, and $\phi$.  
As described in **H.2.4**, this only works for a **linear relationship** for $R(V,I,\phi)$, $X(V,I,\phi)$, and/or $Z(V,I,\phi)$.  
Also, this requires that we have the same number of individual measurements for $V$, $U$, and $\phi$.

### Export
[[Top](#Content)]  


In [78]:
# Export environment to
!mamba env export > environment.yml
# Create via: mamba env create -f environment.yml

In [79]:
# Export notebook to html for quick inspection in a browser.
# https://nbconvert.readthedocs.io/en/latest/usage.html
!jupyter nbconvert --to html --theme light gum-uncertainty-calculation-python.ipynb

[NbConvertApp] Converting notebook gum-uncertainty-calculation-python.ipynb to html
[NbConvertApp] Writing 417642 bytes to gum-uncertainty-calculation-python.html


### Markdown Snippets
[[Top](#Content)]  
[Source](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd)

General:  
`variable` **bold** *italics* ***Bold'italic*** ~~strikethrough~~ [Link](https://en.wikipedia.org)  
> Blockquote
***

Latex formulas (`mhchem` is also supported. see [list](https://docs.mathjax.org/en/latest/input/tex/extensions/index.html)):  
$\textrm{e}^{i\uppi}=-1$  
$\ce{YBa2Cu3O_{7-\delta}}$

List:
1. One
2. Two
3. Three

Unordered list:
- One
- Two
- Three

Task list:
- [x] Some task
- [ ] Todo

Codeblock:
```python
import numpy as np
```

Colors:  
<span style="color:blue">Blue</span>
<span style="color:red">Red</span>
<span style="color:green">Green</span>
<span style="color:magenta">Magenta</span>
<span style="color:#20B2AA">Hex: #20B2AA</span>

Fonts:  
<span style="font-family:Comic Sans MS">Comic Sans</span>

Boxes:  
<div class="alert alert-block alert-info"><b>Tip:</b> Use blue boxes (alert-info) for tips and notes.</div>
<div class="alert alert-block alert-warning"><b>Example:</b> Use yellow boxes for examples that are not inside code cells. or use for mathematical formulas if needed. Typically also used to display warning messages.</div>
<div class="alert alert-block alert-success"><b>Success:</b> This alert box indicates a successful or positive action.</div>
<div class="alert alert-block alert-danger"><b>Danger:</b> This alert box indicates a dangerous or potentially negative action.</div>